In [1]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import gzip
from datetime import datetime, timedelta
from statistics import mean, median
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow
import tensorflow.keras as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization, ReLU, LSTM, Conv1D, Conv2D
from tensorflow.keras.activations import sigmoid, tanh
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm
import csv
import random

from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import precision_score as precision
from sklearn.metrics import recall_score as recall
from sklearn.metrics import f1_score as f1

In [2]:
def retrieve_data(varname, filename):
    df = pd.read_csv(filename, index_col=0)
    df["Date"] = pd.to_datetime(df["Date"])
    return df

def create_classification_data(df, lookback):
    rows = []
    columns = ['Date', 'SP500_relative_change_perc_1'] # date and target
    for i in range(1, lookback + 1):
        new_columns = df.columns.tolist()[1:]
        for x in range(len(new_columns)):
            new_columns[x] = new_columns[x] + "_t-" + str(i)
        columns = columns + new_columns
        
    for i, row in enumerate(df.iterrows()):
        if i > lookback:
            new_row = [row[1][0], row[1][2]]
            for x in range(1, lookback + 1):
                add_row = df.iloc[i - x].tolist()
                new_row = new_row + add_row[1:]
            rows.append(new_row)
    df2 = pd.DataFrame(rows)
    df2.columns = columns
    return df2

def create_train_val_test(df, year_val, year_test, perc_train=None):
    if perc_train == None:
        df["Date"] = pd.to_datetime(df["Date"])
        
        val = df[df['Date'].dt.year == year_val]
        test = df[df['Date'].dt.year == year_test]
        train = df[df['Date'].dt.year < year_val]
#         train = train[train['Date'].dt.year != year_test]
    else:
        train = df.head(round(len(df) * perc_train))
        val = df.tail(len(df) - len(train))
        test = val.tail(round(0.5 * len(val)))
        val = df.head(len(val) - len(test))
    
    y_train = train['SP500_relative_change_perc_1']
    x_train = train.drop(['SP500_relative_change_perc_1'], axis=1)
    
    y_val = val['SP500_relative_change_perc_1']
    x_val = val.drop(['SP500_relative_change_perc_1'], axis=1)
    
    y_test = test['SP500_relative_change_perc_1']
    x_test = test.drop(['SP500_relative_change_perc_1'], axis=1)
    
    return x_train, y_train, x_val, y_val, x_test, y_test

def scale_data(x):
    standard_scaler = MinMaxScaler()
    x = x.drop(["Date"], axis=1)
    x_scaled = pd.DataFrame(standard_scaler.fit_transform(x), columns=x.columns)
    return x_scaled

In [3]:
lookback = 5
val_year = 2018
test_year = 2019

files = {
    # varname: filename
    "S&P500": "Dataset v3/SP500_reduced_data_20220425.csv",
}

for file in files:
    df = retrieve_data(file, files[file])

df = create_classification_data(df, lookback)

display(df)

x_train, y_train, x_val, y_val, x_test, y_test = create_train_val_test(df, val_year, test_year)

,Date,SP500_relative_change_perc_1,SP500_relative_change_perc_1_t-1,SP500_F_relative_change_perc_1_t-1,Gold_F_relative_change_perc_1_t-1,Silver_F_relative_change_perc_1_t-1,Copper_F_relative_change_perc_1_t-1,SP500_williams_R_5_t-1,SP500_williams_R_10_t-1,SP500_williams_R_20_t-1,...,SP500_williams_R_50_t-5,SP500_AD_MACD_12_26_t-5,SP500_stochastic_D_5_5_t-5,SP500_momentum_8_t-5,SP500_momentum_16_t-5,SP500_AD_oscillator_t-5,SP500_stochastic_K_5_t-5,SP500_stochastic_K_10_t-5,SP500_stochastic_K_20_t-5,SP500_stochastic_K_50_t-5
0,2009-07-10,-0.001023,0.001589,0.001589,0.009590,0.006542,0.036778,4.436036,1.937414,3.077523,...,-1.633192,0.644674,0.665849,3.38,-42.73,-0.084206,0.000000,0.175569,0.112216,0.504802
1,2009-07-13,0.024329,-0.001023,-0.001023,0.001647,-0.014275,0.006865,1.696616,1.681511,4.485166,...,-2.632032,-0.704942,0.538816,3.62,-46.17,0.186084,0.271291,0.271291,0.176900,0.472917
2,2009-07-14,0.005551,0.024421,0.024329,0.011739,0.009475,-0.003153,-0.188511,2.335563,4.252820,...,-0.515931,-3.165467,0.346932,-19.91,-65.18,-0.006427,0.021158,0.021158,0.014417,0.310787
3,2009-07-15,0.024720,0.005629,0.005551,0.002499,0.001949,0.018230,-0.387788,1.875529,1.605702,...,-0.756700,-5.174421,0.243015,-40.70,-44.16,0.330092,0.163578,0.163578,0.117823,0.297315
4,2009-07-16,0.011288,0.024754,0.024720,0.010981,0.006710,0.033406,-2.624714,-0.155685,-0.285366,...,-1.048387,-6.440531,0.142669,-36.22,-29.29,0.882040,0.257319,0.213419,0.153722,0.259911
5,2009-07-17,-0.000213,0.011364,0.011288,-0.003517,0.002273,0.002105,-5.619554,-0.794595,-1.521146,...,0.116224,-7.642292,0.209404,-48.10,-31.58,0.082714,0.333673,0.156709,0.127436,0.144569
6,2009-07-20,0.009553,-0.000191,-0.000213,0.000320,0.010407,0.019426,-5.279740,-0.771152,-1.318799,...,-0.611689,-6.748147,0.355146,-18.27,-17.32,0.851924,1.000000,0.506869,0.433885,0.387773
7,2009-07-21,0.002731,0.009617,0.009553,-0.005869,0.012639,0.008610,-1.946366,-4.240291,-2.129318,...,-1.763561,-5.588596,0.550914,-17.49,-15.39,0.512848,1.000000,0.583387,0.583387,0.420205
8,2009-07-22,0.000734,0.002742,0.002731,-0.002108,-0.011519,-0.004888,-5.218415,-2.998393,-1.452770,...,-0.866969,-2.475345,0.714044,36.26,39.64,1.181092,0.979228,0.980350,0.980350,0.729030
9,2009-07-23,0.023268,0.000703,0.000734,0.004003,0.005507,0.035788,-0.845378,-3.848398,-5.633774,...,-3.189189,0.634984,0.853529,42.02,45.64,0.683222,0.954743,0.956860,0.956860,0.821770


In [4]:
def label_data(y):
    positives = []
    negatives = []
    y = list(y)
    for dev in y:
        if dev >= 0:
            positives.append(dev)
        else:
            negatives.append(dev)
    med_pos = median(positives)
    med_neg = median(negatives)
    
    labels = []
    for dev in y:
        if dev >= 0:
            if dev >= med_pos:
                labels.append(2)
            else:
                labels.append(1)
        else:
            if dev <= med_neg:
                labels.append(-2)
            else:
                labels.append(-1)
    return labels

y_train = label_data(y_train)
y_val = label_data(y_val)
y_test = label_data(y_test)

In [5]:
def random_baseline(y):
    counts = [0, 0, 0, 0]
    for i in y:
        if i == -2:
            counts[0] = counts[0] + 1
        elif i == -1:
            counts[1] = counts[1] + 1
        elif i == 1:
            counts[2] = counts[2] + 1
        elif i == 2:
            counts[3] = counts[3] + 1
    print(f"\tDistribution: {counts}")
    print(f"\tRandom baseline accuracy (majority class): {counts[np.argmax(np.asarray(counts))]/ len(y)}")
    
print("Random baseline training set")
random_baseline(y_train)
print("Random baseline validation set")
random_baseline(y_val)
print("Random baseline test set")
random_baseline(y_val)

Random baseline training set
	Distribution: [480, 480, 587, 588]
	Random baseline accuracy (majority class): 0.2754098360655738
Random baseline validation set
	Distribution: [62, 62, 63, 64]
	Random baseline accuracy (majority class): 0.2549800796812749
Random baseline test set
	Distribution: [62, 62, 63, 64]
	Random baseline accuracy (majority class): 0.2549800796812749


In [6]:
train_date = x_train[['Date']]
x_train = x_train.drop(['Date'], axis=1)

val_date = x_val[['Date']]
x_val = x_val.drop(['Date'], axis=1)

test_date = x_test[['Date']]
x_test = x_test.drop(['Date'], axis=1)

In [7]:
x_train = np.asarray(x_train)
x_val = np.asarray(x_val)
x_test = np.asarray(x_test)

y_train = np.asarray(y_train)
y_val = np.asarray(y_val)
y_test = np.asarray(y_test)



x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_val = x_val.reshape((x_val.shape[0], 1, x_val.shape[1]))
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))

y_train = to_categorical(y_train, 4)
y_val = to_categorical(y_val, 4)
y_test = to_categorical(y_test, 4)

y_train = y_train.reshape((y_train.shape[0], 1, y_train.shape[1]))
y_val = y_val.reshape((y_val.shape[0], 1, y_val.shape[1]))
y_test = y_test.reshape((y_test.shape[0], 1, y_test.shape[1]))

In [8]:
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

(2135, 1, 90) (2135, 1, 4)
(251, 1, 90) (251, 1, 4)
(252, 1, 90) (252, 1, 4)


In [41]:
class LSTM_model(object):
    def __init__(self, x, activation_functions, batch_sizes):
        global x_train
        global y_train
        global x_val
        global y_val
        global epochs
        
#         print(f"x {x}")
        
        self.x_train = x_train
        self.x_val = x_val
        
        self.activation_function = activation_functions[x[0]]
        
        self.lstm1 = x[1]
        self.lstm2 = x[2]
        
        self.dense1 = x[3]
        self.dense2 = x[4]

        self.dropout1 = x[5]
        self.dropout2 = x[6]
        
        self.epochs = x[7]
        self.batch_size = batch_sizes[x[8]]
        
    def fit(self):
        model = Sequential()
        model.add(LSTM(self.lstm1, dropout=self.dropout1, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
        if self.lstm2 > 0:
            model.add(LSTM(self.lstm2, dropout=self.dropout2, return_sequences=True))
        if self.dense1 > 0:
            model.add(Dense(self.dense1, activation=self.activation_function))
        if self.dense2 > 0:
            model.add(Dense(self.dense2, activation=self.activation_function))
        model.add(Dense(4, activation='softmax'))
        
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["acc"])
        history = model.fit(x_train, y_train, epochs=self.epochs, batch_size=self.batch_size, verbose=0, validation_data=(x_val, y_val), shuffle=False)
        
        val_loss = mean(history.history['val_acc'][-5:])
#         val_loss = history.history['val_acc'][-1]
        return val_loss

In [42]:
def calculate_fitness(val_loss):
    fitness = val_loss
    return fitness

In [43]:
class EA(object):
    def __init__(self, population_size, activation_functions, batch_sizes):
        self.population_size = population_size
        self.a = 0.2
        self.activation_functions = activation_functions
        self.batch_sizes = batch_sizes
        
    def evaluate(self, x):
        """
        include in fitness function
            relative difference between train_loss and val_loss (smaller is better)
            number of layers (smaller is better)
            bottleneck size (smaller is better)
            val_loss (smaller is better)
        
        """
        lstm = LSTM_model(x, self.activation_functions, self.batch_sizes)
        val_loss = lstm.fit()
        fitness = calculate_fitness(val_loss)
        return fitness
    
    def select_triple(self, candidate, population):
        # select three random instances for differential evolution
        x1, x2, x3 = np.random.choice(range(len(population))), np.random.choice(range(len(population))), np.random.choice(range(len(population)))
        while candidate == x1 or candidate == x2 or candidate == x3 or x1 == x2 or x2 == x3 or x1 == x3:
            # keep selecting new ones until candidate != x1 != x2 != x3
            x1, x2, x3 = np.random.choice(range(len(population))), np.random.choice(range(len(population))), np.random.choice(range(len(population)))
        return population[x1], population[x2], population[x3]
    
    def mutate(self, x1, x2, x3):
        mutated = x1 + (self.a * (x3 - x2))
#         print(f"mutated {mutated}")
        # activation function
        mutated[0] = round(mutated[0])
        mutated[0] = min(mutated[0], len(self.activation_functions) - 1)
        mutated[0] = max(0, mutated[0])
        
        # lstm layer 1
        mutated[1] = round(mutated[1])
        mutated[1] = max(1, mutated[1]) # must be at least one

        # lstm layer 2
        mutated[2] = round(mutated[2])
        mutated[2] = max(0, mutated[2])

        # dense layer 1
        mutated[3] = round(mutated[3])
        mutated[3] = max(4, mutated[3])
        
        # dense layer 2
        mutated[4] = round(mutated[4])
        mutated[4] = max(4, mutated[4])

        # dropout lstm layer 1
        mutated[5] = max(0.05, mutated[5])
        mutated[5] = min(0.95, mutated[5])
        mutated[5] = round(mutated[5],2)
        
        # dropout lstm layer 2
        mutated[6] = max(0.05, mutated[6])
        mutated[6] = min(0.95, mutated[6])
        mutated[6] = round(mutated[6],2)
        
        # epochs
        mutated[7] = round(mutated[7])
        mutated[7] = max(6, mutated[7])

        # batch size
        mutated[8] = round(mutated[8])
        mutated[8] = min(mutated[8], len(self.batch_sizes) - 1)
        mutated[8] = max(0, mutated[8])
        return mutated
        
    def recombine(self, candidate, mutation):
        for i in range(candidate.shape[0]):
            prob = np.random.randint(0, 2)
            if prob == 1:
                candidate[i] = mutation[i]
#         print(f"candidate {candidate}")
        return candidate

    def select(self, x_new, f_new, x_old, f_old):
        x_cat = np.concatenate([x_new, x_old], 0)
        f_cat = np.concatenate([f_new, f_old])
        ind = np.argsort(f_cat)
        x = x_cat[ind]
        f = f_cat[ind]
        return x[-self.population_size:], f[-self.population_size:]
    
    def step(self, x_old, f_old):
        x = np.copy(x_old)
        f = np.copy(f_old)
        for i in tqdm(range(self.population_size), total=self.population_size):
            # choose candidate
            candidate = x[i]
            # select 3 instances for differential evolution
            x1, x2, x3 = self.select_triple(i, x)
            # mutate 3 instances
            mutated_triple = self.mutate(x1, x2, x3)
            # recombine candidate with mutation
            candidate = self.recombine(candidate, mutated_triple)
            x[i] = candidate
            # evaluate candidate solution
            f_candidate = self.evaluate(candidate)
            f[i] = f_candidate
        # select survivors
        x, f = self.select(x, f, x_old, f_old)
        return x, f

In [44]:
def init_population(population_size, activation_functions, batch_sizes, variables):
    # generate initial population
    population = []
    print("Creating initial population...")
    for i in tqdm(range(population_size), total=population_size):
        activation_function = random.randint(0, len(activation_functions) - 1)
        
        lstm1 = random.randint(1, 1.5 * variables)
        lstm2 = random.randint(0, 1.5 * variables)
        dense1 = random.randint(4, 1.5 * variables)
        dense2 = random.randint(4, 1.5 * variables)
        
        dropout1 = round(random.uniform(0.05, 0.95),2)
        dropout2 = round(random.uniform(0.05, 0.95),2)

        epochs = random.randint(10, 500)
        
        batch_size = random.randint(0, len(batch_sizes) - 1)
    
        population.append(np.asarray([activation_function, lstm1, lstm2, dense1, dense2, dropout1, dropout2, epochs, batch_size], dtype='object'))
    print("Initial population ready")
    return np.asarray(population)

def evaluate_init_population(ea, x):
    # evaluate initial population
    f = []
    print("Evaluating initial population...")
    for i in tqdm(range(x.shape[0]), total=x.shape[0]):
        instance = x[i]
        f.append(ea.evaluate(instance))
    print("Evaluation initial population completed")
    return np.asarray(f)

def print_best(x, activation_functions, batch_sizes, fitness):
    print(f"\nMost suitable parameters -- Accuracy of {fitness}:")
    print(f"\tActivation function:           \t{activation_functions[x[0]]}")
    print(f"\tLSTM nodes layer 1:            \t{x[1]}")
    print(f"\tLSTM nodes layer 2:            \t{x[2]}")
    print(f"\tDense nodes layer 1:           \t{x[3]}")
    print(f"\tDense nodes layer 2:           \t{x[4]}")
    print(f"\tDropout LSTM layer 1:          \t{x[5]}")
    print(f"\tDropout LSTM layer 2:          \t{x[6]}")
    print(f"\tEpochs trained:                \t{x[7]}")
    print(f"\tBatch Size:                    \t{batch_sizes[x[8]]}")

def plot_convergence(f_best):
    fig1 = make_subplots(rows=1, cols=1, specs=[[{'type':'xy'}]])
    
    x_values = []
    for i in range(len(f_best)):
        x_values.append(i)
    fig1.add_trace(go.Scatter(x=x_values, y=f_best, mode="lines"), row=1, col=1)

    fig1.update_layout(
        title = f'Validation Accuracy Over Autoencoder Tuning Generations', 
        xaxis1 = dict(title_text = 'Generation'),
        yaxis1 = dict(title_text = "Validation Accuracy")
    )
    fig1.write_image("Plots/opt lstm 1 - lookback 5.png")
    fig1.show()

def validate_best(x, ea):
    print("\nValidating solution...")
    ea.evaluate(x)
    print("Solution validated")

In [45]:
"""
    instance = [activation function, --> one of list
        nodes layer 1, --> integer, not 0
        nodes layer 2, --> integer, may be 0
        nodes layer 3, --> integer, may be 0
        batch_normalization, --> binary integer
        batch_size --> one of list
        ]
"""

population_size = 30
generations = 20
activation_functions = ['sigmoid', 'tanh']
batch_sizes = [64, 128, 256, 512]

variables = x_train.shape[2]

ea = EA(population_size, activation_functions, batch_sizes)
x = init_population(population_size, activation_functions, batch_sizes, variables)
f = evaluate_init_population(ea, x)

populations = []
populations.append(x)
f_best = [f.max()]

start_time = datetime.now()

print("--> STARTING EVOLUTION")
early_stop = 0
for i in range(generations):
    print(f'Generation: {i}\tBest fitness: {f.max()}')
    x, f = ea.step(x, f)
    print(x)
    populations.append(x)

    if f.max() > f_best[-1]:
        f_best.append(f.max())
        early_stop = 0
    else:
        f_best.append(f_best[-1])
        early_stop += 1
    if early_stop == 3:
        print("Early stop triggered at generation {i} after not improving fitness for three generations")
        break
print("--> EVOLUTION FINISHED")

end_time = datetime.now()
evolution_time = end_time - start_time
evolution_time_seconds = evolution_time.total_seconds()
print(f"\nElapsed time in minutes: {evolution_time_seconds/60}")

print(f)
print(f.min())
index_best_parameters = np.where(f == f.max())[0][0]
print(index_best_parameters)
print_best(x[index_best_parameters], activation_functions, batch_sizes, f.max())
validate_best(x[index_best_parameters], ea)
plot_convergence(f_best)

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

Creating initial population...
Initial population ready
Evaluating initial population...


  0%|                                                                                           | 0/30 [00:12<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
plot_convergence(f_best)